## Rubin RSP Zeropoint Notebook
The aims of this notebook are as follows:
1) To extract zeropoints from random single exposures and coadds in the RSP images.
2) To verify that the pixel ratios of single exposures and coadds match expectations, given their different zeropoints.

We start by importing packages. Note lsst_science_pipeline needs to be downloaded from Github. 

In [ ]:
from lsst.rsp import get_tap_service, retrieve_query
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler
import lsst.geom
import matplotlib.pyplot as pl
from matplotlib.colors import LogNorm
import numpy as np
from photutils.aperture import CircularAperture
from skimage.feature import peak_local_max
# The following package is located here: https://github.com/LSST-strong-lensing/slsim/blob/main/slsim/lsst_science_pipeline.py
from lsst_science_pipeline import tap_query,list_of_calexp,warp_to_exposure
from Util.param_util import random_ra_dec,random_radec_string
service = get_tap_service("tap")
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')
skymap = butler.get("skyMap")

We will first select a random coordinate patch within the LSST survey field, then retrieve the information regarding single exposure images within some given radius around that coordinate.

In [ ]:
np.random.seed(1)
random_ra_dec = random_radec_string(55,70,-43,-30,1)
ra_i = float(random_ra_dec[0].split(',')[0])
dec_i = float(random_ra_dec[0].split(',')[1])
print('RA/Dec:',random_ra_dec)

# Retrieving details of the single exposures around that coordinate:
expo_information = tap_query(random_ra_dec[0],radius = str(0.1))
expo_information.to_pandas()

Next, we'll find the coadd corresponding to that random coordinate:

In [ ]:
# Retrieving the corresponding coadd for the above coordinates: we first need to identify the tract and patch the coadd belongs to.
my_spherePoint = lsst.geom.SpherePoint(ra_i*lsst.geom.degrees,
                                       dec_i*lsst.geom.degrees)
tract = skymap.findTract(my_spherePoint)
patch = tract.findPatch(my_spherePoint)
my_tract = tract.tract_id
my_patch = patch.getSequentialIndex() 
# Looking for the i-band coadd here:
coadd_im = butler.get("deepCoadd",dataId={"tract": (my_tract),"patch": (my_patch), "band": "i"})
# This retrieves the exposure map, i.e. the number of single exposure images which make up a given pixel (note, this is number of exposures, 
# not total exposure time) 
coadd_n_exp =  butler.get("deepCoadd_nImage",dataId={"tract": (my_tract),"patch": (my_patch), "band": "i"})

We want to compare pixel values between the coadd and the single exposures. To do this, they first need to be warped to the same pixel grid:

In [ ]:
#Retrieves list of single-exposure images corresponding to (the first 5 extracts from) the above query:
n_exp = 5
list_of_sing_exp = list_of_calexp(expo_information[0:n_exp],butler)

def aligned_calexp_to_ref(calexp_image,ref):
    """Aligns list of given images to the reference image provided

    :param calexp_image: list of calexp images.
    :param ref: reference image used to warp to
    :return: list of aligned images.
    """
    aligned_calexp_image = []
    for i in range(len(calexp_image)):
        aligned_calexp_image.append(warp_to_exposure(calexp_image[i], ref))
    return aligned_calexp_image

list_of_warps = aligned_calexp_to_ref(list_of_sing_exp,coadd_im)  
#We'll pick the first one in the list for the moment:
single_exposure = list_of_sing_exp[0];rand_warp = list_of_warps[0]

We will now find the zeropoints of the single exposure and coadd image. This is done as follows:

In [ ]:
ZP_sing_exp = list_of_sing_exp[0].getPhotoCalib().instFluxToMagnitude(1)
ZP_coadd_exp = coadd_im.getPhotoCalib().instFluxToMagnitude(1)
print(f'Single Exposure ZP: {ZP_sing_exp}, Coadd ZP: {ZP_coadd_exp}') 

We'll now plot one of the single exposure images and its corresponding coadd on the same pixel grid. We will plot this for the *calibrated* images. These are all in units of nano-Jansky. 

In [ ]:
calib_sing_exposure = single_exposure.getPhotoCalib().calibrateImage(single_exposure.getMaskedImage()).image.array
calib_rand_coadd = coadd_im.getPhotoCalib().calibrateImage(coadd_im.getMaskedImage()).image.array
calib_rand_warp = rand_warp.getPhotoCalib().calibrateImage(rand_warp.getMaskedImage()).image.array

fig,ax = pl.subplots(1,2,figsize=(10,5))
ax[0].imshow(coadd_im.image.array,norm=LogNorm())
ax[0].set_title('Coadd',fontsize=12)
ax[1].imshow(calib_rand_warp,norm=LogNorm())
ax[1].set_title('Single Exposure',fontsize=12)
pl.suptitle(f'Median Pixel Values (in nJy):\n'
            f'Sing-exp: {str(np.round(np.nanmedian(calib_rand_warp),5))}, '+\
            f'Coadd: {str(np.round(np.nanmedian(calib_rand_coadd),5))}, '+\
            f'Per-pixel Ratio: {str(np.round(np.nanmedian(calib_rand_coadd/calib_rand_warp),5))}')
pl.tight_layout()
pl.show()

In [ ]:
def compare_calibrated_images(coadd,sing_exp,calib_coadd,calib_sing_exp):
    """
    This function locates the brightest objects in the coadd image. Aperture photometry is then performed for these same positions in both the single
    exposure and the coadd (on both calibrated and uncalibrated images). The ratio of fluxes should be equal to the ratio of zeropoints in the uncalibrated 
    case and equal to unity in the calibrated case.
    Just taking the pixel ratios of the whole image produces the wrong result, as the ratios become dominated by noise in the pixels, hence only the 
    brightest pixels are used here.
    :param coadd: Numpy array containing the coadd image values
    :param sing_exp: Numpy array containing the single exposure values
    :param calib_coadd: Numpy array containing the coadd image values, in units of nJy.
    :param calib_sing_exp: Numpy array containing the single exposure values, in units of nJy.
    :return: None
    """
    x,y = peak_local_max(calib_coadd,threshold_rel=0.1,min_distance=100).T
    apertures = CircularAperture(np.array([y,x]).T,10)
    photometry_calib_coadd = apertures.do_photometry(calib_coadd)[0]
    photometry_calib_sing_exp = apertures.do_photometry(calib_sing_exp)[0]
    photometry_coadd = apertures.do_photometry(coadd)[0]
    photometry_sing_exp = apertures.do_photometry(sing_exp)[0]
    #Now plotting the image with aperture positions overlaid, and the flux ratios of the coadd/single-exposure images. 
    fig,ax = pl.subplots(1,3,figsize=(15,4))
    ax[0].imshow(calib_coadd,norm=LogNorm(),origin='lower')
    ax[0].scatter(y,x,c='k',marker='x',alpha=0.5)
    ax[1].hist(photometry_calib_coadd/photometry_calib_sing_exp)
    ax[1].set_title('Calibrated')
    ax[1].set_xlabel('Pixel Ratio of Coadd/Sing-exp (nJy/nJy)')
    ax[1].set_ylabel('Counts')
    ax[2].hist(photometry_coadd/photometry_sing_exp)
    ax[2].set_xlabel('Pixel Ratio of Coadd/Sing-exp')
    ax[2].set_ylabel('Counts')
    ax[2].set_title('Uncalibrated')
    ax[2].scatter(10**(-(31.846-27)/2.5),1,c='k',label='ZP Ratio (Flux Units)')
    ax[2].legend()
    pl.show()

compare_calibrated_images(coadd_im.image.array,rand_warp.image.array,calib_rand_coadd,calib_rand_warp)

As expected, the calibrated images have pixel ratios ~1, and the single exposures have pixel ratios roughly equal to the ratio of zeropoints (in flux units).